In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.CodeMirror {font-family:Consolas; font-size:22pt;}
div.output {font-size:22pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:22pt;}
div.prompt {min-width:70px;}
</style>
"""))

<b><font size="6" color="red">ch10. 데이터베이스 연동</font></b>

## 1절. SQLite 데이터 베이스연결

- SQLite 데이터 베이스는 별도의 DBMS 없이 SQL을 이용하여 DB 엑세스할 수 있도록 만든 **간단한 디스크 기반의 데이터베이스를 제공**하는 C 라이브러리
- SQLite는 프로토타입(시제품 나오기전까지 제품)을 만들때 사용. 정식 제품은 오라클이나 다른 DBMS를 이식
- [DB browser for SQLite](https://sqlitebrowser.org/dl)

## 1.1 SQLite browser 설치 및 sqlite3 패키지 load

In [3]:
import sqlite3
sqlite3.sqlite_version

'3.35.4'

In [4]:
import pandas as pd
pd.__version__

'1.2.4'

## 1.2 데이터 베이스 연결

In [5]:
# DB 연결 생성시, 파일이 있으면 그 파일 연결. 파일이 없으면 빈 파일을 생성
conn = sqlite3.connect('data/ch10_example.db')
conn

In [6]:
# 커서 객체 반환. 커서는 SQL문 실행시키고 결과데이터(SELECT)를  조회하는데 사용객체
cursor = conn.cursor()
cursor

In [7]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
""")

In [11]:
cursor.execute("DROP TABLE MEMBER")

In [12]:
cursor.execute("""
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
""")

In [14]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
cursor.execute("INSERT INTO MEMBER VALUES ('김길동', 23, 'k@h.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('신길동', 30, 's@h.com')")

In [16]:
conn.commit() # 反 conn.rollback() DML문에서만 commit

In [18]:
# select 실행결과는  cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [19]:
# select 문의 결과를 받는 함수
## fetchall() - 결과를 모두 받을 때(튜플 list)
## fetchone() - 결과를 한행씩 받을 때(튜플)
## fetchmany(n) - 결과를 n행 받을 때
print(cursor.fetchall())

[('김길동', 23, 'k@h.com'), ('신길동', 30, 's@h.com'), ('홍길동', 20, 'h@h.com')]


In [20]:
print(cursor.fetchall()) # 한번 소요된 cursor객체는 다시 fetch할 수 없음

[]


In [22]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()
members

[('홍길동', 20, 'h@h.com'), ('김길동', 23, 'k@h.com'), ('신길동', 30, 's@h.com')]

In [23]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('김길동', 23, 'k@h.com')
('신길동', 30, 's@h.com')


In [25]:
cursor.execute("SELECT * FROM MEMBER")
member_list = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    member_list.append({'name':member[0], 'age':member[1], 'email':member[2]})
    print('이름 :',member[0], '\t나이 :',member[1], '\t메일 :',member[2])

이름 : 홍길동 	나이 : 20 	메일 : h@h.com
이름 : 김길동 	나이 : 23 	메일 : k@h.com
이름 : 신길동 	나이 : 30 	메일 : s@h.com


In [26]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '김길동', 'age': 23, 'email': 'k@h.com'},
 {'name': '신길동', 'age': 30, 'email': 's@h.com'}]

In [29]:
cursor.execute('SELECT * FROM MEMBER ORDER BY AGE DESC')
for member in cursor.fetchmany(10):
    print(member[0],member[1], member[2])

신길동 30 s@h.com
김길동 23 k@h.com
홍길동 20 h@h.com


In [30]:
cursor.close() # cursor.close는 생략 가능
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기
- qmark, named(추천) 방법

In [31]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','유낄똥')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com')]

In [33]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1은 ?')
name2 = input('검색할 이름2는 ?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", ('홍길동','유낄똥'))
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", (name1,name2))
cursor.fetchall()

검색할 이름1은 ?홍길동
검색할 이름2는 ?유낄동


[('홍길동', 20, 'h@h.com')]

In [35]:
name1 = input('검색할 이름1은 ?')
name2 = input('검색할 이름2는 ?')
names = [name1, name2] # 튜플이나 리스트형 변수
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", names)
cursor.fetchall()

검색할 이름1은 ?홍길동
검색할 이름2는 ?김길동


[('홍길동', 20, 'h@h.com'), ('김길동', 23, 'k@h.com')]

In [40]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름은 ?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME =  :name ", {'name':name1})
members = cursor.fetchall()
if len(members) == 0:
    print("해당 이름의 데이터가 없습니다")
else:
    print(members)
    # print(cursor.fetchall()) # 무조건 한행도 출력 안 됨

검색할 이름은 ?홍길똥
해당 이름의 데이터가 없습니다


In [41]:
# member 테이블에 입력 (사용자로부터 이름, 나이, 메일을 입력받아 DB에 insert하는 프로그램 수행)
try:
    name = input('입력할 이름은 ?')
    age  = int(input('입력할 나이는 ?'))
    email = input('입력할 메일은 ? ')
except:
    print('유효하지 않는 나이 입력시 18세로 초기화')
    age = 18
    email = input('입력할 메일은 ?')
inputdata = {'name':name, 'age':age, 'email':email}
# cursor.execute("INSERT INTO MEMBER VALUES (?, ?, ?)", (name, age, email))
# cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", inputdate)
cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", 
                      {'name':name, 'age':age, 'email':email})
conn.commit()

입력할 이름은 ?이길동
입력할 나이는 ?23
입력할 메일은 ? k@l.com


In [47]:
class User:
    '이름, 나이, 메일이 있는 class'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email = email
    def __str__(self):
        return '이름:{:4}\t나이:{:3}\t메일:{}'.format(self.name, self.age, self.email)
    def as_dict(self):
        return {'name':self.name, 'age':self.age, 'email':self.email}

try:
    name = input('입력할 이름은 ?')
    age  = int(input('입력할 나이는 ?'))
    email = input('입력할 메일은 ? ')
except:
    print('유효하지 않는 나이 입력시 18세로 초기화')
    age = 18
    email = input('입력할 메일은 ?')
user = User(name, age, email)
cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", user.as_dict())
conn.commit()

입력할 이름은 ?유유유
입력할 나이는 ?23
입력할 메일은 ? t


In [45]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

[('홍길동', 20, 'h@h.com'),
 ('김길동', 23, 'k@h.com'),
 ('신길동', 30, 's@h.com'),
 ('이길동', 23, 'k@l.com'),
 ('윤길동', 26, 'y@hong.com')]

In [46]:
# member 테이블에 입력(0을 입력할 때까지 계속 입력)
user_list = [] # 입력한 user 객체들이 저장될 예정
while True:
    try:
        name = input('입력할 이름은 ?(입력을 종료하고자 하면 0입력)')
        if name == '0':
            break
        age  = int(input('입력할 나이는 ?'))
        email = input('입력할 메일은 ? ')
    except:
        print('유효하지 않는 나이 입력시 18세로 초기화')
        age = 18
        email = input('입력할 메일은 ?')
    user = User(name, age, email)
    user_list.append(user)
    cursor.execute("INSERT INTO MEMBER VALUES (:name, :age, :email)", user.as_dict())
conn.commit()
print('입력된 데이터는 ', len(user_list), '명 추가되었습니다')
for user in user_list:
    print(user)

입력할 이름은 ?(입력을 종료하고자 하면 0입력)홍낄똥
입력할 나이는 ?24
입력할 메일은 ? K@naver.com
입력할 이름은 ?(입력을 종료하고자 하면 0입력)유낄똥
입력할 나이는 ?24
입력할 메일은 ? y@hong.com
입력할 이름은 ?(입력을 종료하고자 하면 0입력)0
입력된 데이터는  2 명 추가되었습니다
이름:홍낄똥 	나이: 24	메일:K@naver.com
이름:유낄똥 	나이: 24	메일:y@hong.com


In [49]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
[cx_Oracle](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)

In [5]:
import cx_Oracle
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [6]:
# (1) conn 얻어오는 방법1
conn = cx_Oracle.connect("scott", "tiger", dsn = oracle_dsn)
conn
# 여기서 에러날 시 VC_redist.x64.exe 인스톨

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [56]:
conn.close()

In [2]:
# (2) conn 얻어오는 방법2
conn = cx_Oracle.connect("scott",'tiger', "localhost:1521/xe")
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [3]:
# import sqlite3
# conn = sqlite3.connect('data/ch10_example.db')
# conn

In [7]:
# cursor 얻어오기
cursor = conn.cursor()

In [8]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

In [9]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [10]:
sql = "SELECT * FROM EMP"
cursor.execute(sql)
for e in cursor:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [11]:
sql = "SELECT * FROM EMP WHERE ENAME=:ename"
ename = input('검색할 이름은 ?')
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
if len(emp) == 0:
    print('입력하신 이름의 데이터는 없습니다')
else:
    for e in emp:
        print(e)

검색할 이름은 ?SMITH
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)


In [12]:
# 데이터 베이스 검색 결과를 DataFrame으로 
import pandas as pd
cursor.execute('SELECT * FROM EMP')
data = cursor.fetchall() # 튜플 리스트
data_df = pd.DataFrame(data) # 데이터 프레임
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [13]:
type(data_df)

pandas.core.frame.DataFrame

In [14]:
# 각 필드 특징 정보 (필드명, type, display_size, 정확도, scale, nullable.....)
cursor.description

[('EMPNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 0),
 ('ENAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 10, 10, None, None, 1),
 ('JOB', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 9, 9, None, None, 1),
 ('MGR', <cx_Oracle.DbType DB_TYPE_NUMBER>, 5, None, 4, 0, 1),
 ('HIREDATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1),
 ('SAL', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('COMM', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 7, 2, 1),
 ('DEPTNO', <cx_Oracle.DbType DB_TYPE_NUMBER>, 3, None, 2, 0, 1)]

In [16]:
[row[0] for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [17]:
data_df.columns

RangeIndex(start=0, stop=8, step=1)

In [18]:
data_df.columns = [row[0] for row in cursor.description]
data_df

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [19]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [20]:
cursor.close()
conn.close()

# 3절 연습문제

In [8]:
# 클래스 생성은 선택사항
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def __str__(self):
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format('*'*self.grade,
                    self.name, self.phone, self.email, self.age, self.etc)
    def to_dict(self):
        return {'name':self.name, 'phone':self.phone, 'email':self.email,
                'age':self.age,   'grade':grade, 'etc':etc}

In [9]:
# 1.입력
def insert_member():
    # 사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다
    global conn
    cursor = conn.cursor()
    name = input('이름 : ')
    phone = input('전화번호 : ')
    email = input('이메일 : ')
    try:
        age = int(input("나이: "))
    except:
        print("유효하지 않은 값을 입력해서 20으로 초기화")
        age=20
    try:
        grade = int(input("고객 등급(1~5): "))
        #if grade<1 | grade>5: # 이렇게 하면 안 됨
        #    print('9를 입력해봐 if문이 거짓이라지') 
        if (grade<1) | (grade>5):
            print('고객등급을 1~5사이를 입력하지 않으면 1등급으로 초기화')
            grade = 1
    except:
        print("유효하지 않은 등급을 입력하면 1등급으로 초기화")
        grade = 1
    etc = input('기타 특징 :')
    sql = "insert into member values (:name, :phone, :email, :age, :grade, :etc)"
    cursor.execute(sql, {'name':name, 'phone':phone, 
            'email':email, 'age':age, 'grade':grade, 'etc':etc})
    conn.commit()
    cursor.close()

In [23]:
# 위의 메소드 test
insert_member()

이름 : 홍길동
전화번호 : 010-7777-7777
이메일 : k@hong.com
나이: 34
고객 등급(1~5): 4
기타 특징 :etc


In [10]:
# 2. 전체 출력
def select_all():
    # "SELECT * FROM MEMBER"의 결과를 출력(DataFrame / 리스트)
    global conn
    cursor = conn.cursor()
    import pandas as pd
    sql = "SELECT * FROM MEMBER"
    cursor.execute(sql)
    members = cursor.fetchall()
    # DataFrame을 출력
    members_df = pd.DataFrame(members, columns=[row[0] for row in cursor.description])
    print(members_df)
    # 리스트를 출력
    # for member in members:
    #     print(member[0], member[1], member[2], member[3], member[4], member[5], sep='\t')
    cursor.close()

In [11]:
# 위의 메소드 test
select_all()

  NAME          PHONE          EMAIL  AGE  GRADE ETC
0  홍길동  010-9999-9999  hong@hong.com   20      4  어려
1  김김김  010-7111-7111   kim@hong.com   16      3  이뻐


In [12]:
# 3. 이름 검색
def select_name():
    # 사용자로부터 검색하고자 하는 이름을 받아 결과 출력(DataFrame형태 / 리스트형태)
    global conn
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER WHERE NAME=:name"
    name = input('조회할 이름 : ')
    cursor.execute(sql, {'name':name})
    members = cursor.fetchall()
    if len(members) ==0:
         print("입력한 이름의 데이터가 존재하지 않습니다")
    else:
        import pandas as pd
        # DataFrame을 출력 
        members_df = pd.DataFrame(members, 
                    columns=[row[0] for row in cursor.description])
        print(members_df)
        # 리스트를 출력
        #for member in members:
        #    print(member[0], member[1], member[2], member[3], member[4], member[5], sep='\t')
    cursor.close()

In [13]:
# 위의 메소드 test
select_name()

조회할 이름 : 홍길동
  NAME          PHONE          EMAIL  AGE  GRADE ETC
0  홍길동  010-9999-9999  hong@hong.com   20      4  어려


In [88]:
# 4. 메일 삭제
def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 해당 메일이 있는지 확인하고
    # 해당이메일이 없으면 없다출력. 있으면 삭제하고 삭제했다 출력
    global conn
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER WHERE EMAIL=:email"
    email = input("삭제할 이메일 주소 : ")
    cursor.execute(sql, {'email':email})
    members=cursor.fetchall()
    if len(members)==0:
        print("입력한 이메일의 데이터가 존재하지 않습니다")
    else:
        cursor.execute("DELETE FROM MEMBER WHERE EMAIL=:email", 
                       {'email':email})
        print('삭제되었습니다')
        conn.commit()
    cursor.close()

In [89]:
# 위의 메소드 test
delete_email()

삭제할 이메일 주소 : d
입력한 이메일의 데이터가 존재하지 않습니다


In [90]:
# 지금 현재 DB 내용
cursor = conn.cursor()
cursor.execute("select * from member")
cursor.fetchall()

[('홍길동', '010-9999-9999', 'hong@hong.com', 20, 4, '어려'),
 ('김김김', '010-7111-7111', 'kim@hong.com', 16, 3, '이뻐')]

In [18]:
# 5. CSV 내보내기
def save_csv():
    # 'SELECT * FROM MEMBER'의 결과를 CSV로 내보내기(HEADER 포함)
    global conn
    cursor = conn.cursor()
    import csv
    sql = "SELECT * FROM MEMBER"
    import pandas as pd
    cursor.execute(sql)
    members = cursor.fetchall()
    cursor.close()
    filename = input('파일 이름 : ')
    # 데이터프레임을 csv로
#     members_df = pd.DataFrame(members, 
#             columns=[row[0] for row in cursor.description])
#     members_df.to_csv(filename+'.csv', encoding='cp949')
#     print(filename+'.csv로 저장되었습니다')
    # list를 csv로
    
    with open(filename+'.csv', 'w', encoding='cp949', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([row[0].lower() for row in cursor.description])
#         for member in members:
#             writer.writerow(list(member))
        writer.writerows(members)
    # dictionary를 csv로
    '''
    with open(filename+'.csv', 'w', encoding='cp949', newline='') as f:
        dict_writer = csv.DictWriter(f, fieldnames=[row[0].lower() for row in cursor.description])
        dict_writer.writeheader()
        for m in members:
            dict_writer.writerow({'name':m[0], 'phone':m[1], 'email':m[2],'age':m[3], 'grade':m[4], 'etc':m[5]})
    '''                 

In [19]:
# 위의 메소드 TEST
save_csv()

파일 이름 : 0


In [91]:
# 0. 종료
def close_sql():
    global conn
    conn.close()
    print("종료합니다")

In [92]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",
             sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break

In [93]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data.db')
    main()

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 2
  NAME          PHONE          EMAIL  AGE  GRADE ETC
0  홍길동  010-9999-9999  hong@hong.com   20      4  어려
1  김김김  010-7111-7111   kim@hong.com   16      3  이뻐
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 1
이름을 입력해:마길동
전화번호를 입력해:010-8888-8888
이메일을 입력해:ma@hong.com
나이를 입력해:30
등급을 입력해:3
기타를 입력해:마
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 3
조회할 이름 : 마길동
  NAME          PHONE        EMAIL  AGE  GRADE ETC
0  마길동  010-8888-8888  ma@hong.com   30      3   마
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 4
삭제할 이메일 주소 : ma
입력한 이메일의 데이터가 존재하지 않습니다
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 5
파일 이름 : 0
1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 0
종료합니다
